# Counting number of objects of a 1D image

This is a practice code. The idea is that we take a colummn vector of 0's and 1's and treat 1 a light and 0 as no light. Within this context the number of sequences of consequtive 0's can be be interpreted as the number of objects in an image. I will create a network that will takie (nx1) inputs (a 1D image) apply two filters - one counting the number of left edges and one counting the number of right edges getting a n x 1 x 2 tensor and the stack that into a one last layer with an identity activation. So the network should correctly output the number of objects

There will be no training, I will fix the weights heuristically. Also there will be only a forward method


The plan:

* Make test cases
* Function that will take a d x 1 column vector and return two d x 1 vectors with left and right edges
* Apply ReLU activations, so that we have clean left and right edges vectors - 0 no edge, 1 edge.
* Count the number of right edges + 1 if the las element is a left edge by appying weights and a identity activation in the last layer
* Test the network

In [97]:
# Test cases:
import numpy as np
image1 = np.array([[0,1,0]]).T
image2 = np.array([[1,0,1]]).T
image3 = np.array([[0]])
image4 = np.array([[1]])
image5 = np.array([[1,1,1,1,1]]).T
image6 = np.array([[0,0,0,0,0]]).T
image7 = np.array([[0,1,1,0,0]]).T
image8 = np.array([[0,1,0,1,1]]).T
image9 = np.array([[1,1,0,0,1,0,1,1,1,0,0,0,1]]).T
image10 = np.array([[0,0,1,1,0,0,1,1,0,1,1,1,0,0,0,1,0,1]]).T


expected = {'image1': (image1, 2), 'image2': (image2, 1), 'image3': (image3, 1), 'image4': (image4, 0),
            'image5': (image5, 0), 'image6': (image6, 1), 'image7': (image7, 2), 'image8': (image8, 2),
            'image9': (image9, 3), 'image10': (image10,5)}
    


In [139]:
right_edge = np.array([[-1], [1]])

def convolute(image, f):
    image = np.vstack((np.array([[0]]), image))
    feature_map = []
    for i in range(len(image)-1):
        feature_map.append(np.matmul(f, image[i:i+2]))
    return np.array(feature_map).reshape((len(image)-1,1))


def ReLU(x):
    if type(x) == float or type(x) == int:
        return x if x>0 else 0
    else:
        return np.vectorize(lambda x: x if x>0 else 0)(x)

def is_last_left(image): ## This is a linear transformation with offset one. Basically 1 - last input
    weights = np.zeros(image.shape).T
    weights[:,[-1]] = 1
    return 1 - np.matmul(weights,image)
    
    


In [140]:
for key in expected:
    image, num_objects = expected[key]
    
    right = ReLU(convolute(image, right_edge.T))  #applying filter and ReLU activatio 
    last_left = ReLU(is_last_left(image))   #applying feed forward layer and ReLU activatio 
    
    
    # here we come to the output layer:
    weights_right = np.ones(right.shape)
    weights_right[[0],:] = 0
    weights_last_left = np.ones(last_left.shape)
    result = np.matmul(weights_right.T, right) + np.matmul(weights_last_left.T, last_left)

    # Check results of tests
    result = int(result)
    if result == num_objects:
        test_result = 'passed'
    else:
        test_result ='failed'

    print('For {} the test produces {} objects, while {} where expected. The test {}'
          .format(key, result, num_objects, test_result))



For image1 the test produces 2 objects, while 2 where expected. The test passed
For image2 the test produces 1 objects, while 1 where expected. The test passed
For image3 the test produces 1 objects, while 1 where expected. The test passed
For image4 the test produces 0 objects, while 0 where expected. The test passed
For image5 the test produces 0 objects, while 0 where expected. The test passed
For image6 the test produces 1 objects, while 1 where expected. The test passed
For image7 the test produces 2 objects, while 2 where expected. The test passed
For image8 the test produces 2 objects, while 2 where expected. The test passed
For image9 the test produces 3 objects, while 3 where expected. The test passed
For image10 the test produces 5 objects, while 5 where expected. The test passed
